In [1]:
from datasets import load_from_disk
import datasets
from datasets import load_dataset
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
from transformers import AutoModelForSequenceClassification
from torch.optim import AdamW
from transformers import get_scheduler
import torch
from tqdm.auto import tqdm
import evaluate
import random
import argparse
from nltk.corpus import wordnet as wn
from nltk import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk import pos_tag
import string
import re

/Users/danieldacosta/miniconda3/envs/hack/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def custom_transform(example):
    
    ################################
    ##### YOUR CODE BEGINGS HERE ###
    
    # Design and implement the transformation as mentioned in pdf
    # You are free to implement any transformation but the comments at the top roughly describe
    # how you could implement two of them --- synonym replacement and typos.
    
    # You should update example["text"] using your transformation
        
    raise NotImplementedError
    
    
    ##### YOUR CODE ENDS HERE ######
    
    return example




In [149]:
# Define the directory where the dataset is saved
load_directory = "dataset/"

# Load the dataset from the specified directory
loaded_data = load_from_disk(load_directory)

# Now you can access the individual splits (train, test, unsupervised) as follows:
train_dataset = loaded_data["train"]
test_dataset = loaded_data["test"]
unsupervised_dataset = loaded_data["unsupervised"]

In [150]:
with open("dataset_txt.txt", 'w') as file:
    for sentence in train_dataset['text']:
        file.write(sentence + '\n')


In [57]:
# replacement probability
probability = 0.6

# Generate a random number between 0 and 1
random_number = random.random()

In [60]:
example = test_dataset[15]['text']

In [61]:
example

'Now, I LOVE Italian horror films. The cheesier they are, the better. However, this is not cheesy Italian. This is week-old spaghetti sauce with rotting meatballs. It is amateur hour on every level. There is no suspense, no horror, with just a few drops of blood scattered around to remind you that you are in fact watching a horror film. The "special effects" consist of the lights changing to red whenever the ghost (or whatever it was supposed to be) is around, and a string pulling bed sheets up and down. Oooh, can you feel the chills? The DVD quality is that of a VHS transfer (which actually helps the film more than hurts it). The dubbing is below even the lowest "bad Italian movie" standards and I gave it one star just because the dialogue is so hilarious! And what do we discover when she finally DOES look in the attic (in a scene that is daytime one minute and night the next)...well, I won\'t spoil it for anyone who really wants to see, but let\'s just say that it isn\'t very "novel"

# Synonym

In [16]:
def get_wordnet_pos(treebank_tag): # Convert POS-Tag from treebank_tag to Wordnet tags
    if treebank_tag.startswith('J'):
        return wn.ADJ
    elif treebank_tag.startswith('V'):
        return wn.VERB
    elif treebank_tag.startswith('N'):
        return wn.NOUN
    elif treebank_tag.startswith('R'):
        return wn.ADV
    else:
        return None  # If no match is found, return None
    


In [17]:
def replace_with_synonym(text: str, pct_synonyms: float = 0.4) -> str:
    """Replace {pct_synonyms}% of words in a sentence with their synonyms
    :param text (str):
    :param pct_synonyms (float): percentage of words to be replaced. Between 0 and 1
    :return: transformed text
    """
    # Tokenize sentence
    word_list  = word_tokenize(text)
    n_tokens = len(word_list)
    n_synonyms = int(n_tokens*pct_synonyms)

    # Perform POS tagging to help selecting synonym
    pos_tags = pos_tag(word_list)
    syn_dict = {}
    for i in range(n_synonyms):
        chosen_word = random.choice(word_list)
        synonyms = wn.synsets(chosen_word)

        # If chosen word doesn't have a synonym
        # retry until finding a word that does.
        max_retry = len(word_list)
        retry = 0 
        while (len(synonyms) == 0 and retry <= max_retry):
            chosen_word = random.choice(word_list)
            synonyms = wn.synsets(chosen_word)

            if len(synonyms) > 0:
                # Print the word and its POS tag
                chosen_word_tag = [tag for word, tag in pos_tags if word == chosen_word][0]
                chosen_word_wordnet_tag = get_wordnet_pos(chosen_word_tag)
                
                selected_synonym = chosen_word
                for syn in synonyms:
                    # Only consider synonyms that have the same POS_TAG
                    if syn.pos() == chosen_word_wordnet_tag:
                        for lemma in syn.lemmas():
                            if (lemma.name() != chosen_word):
                                # Select the first synonym that appears
                                selected_synonym = lemma.name().replace('_', ' ') # Replace underscores for multi-word synonyms
                                break
                        break
                    break
                if selected_synonym == chosen_word:
                    synonyms = [] # continue while loop
                else:
                    # print("selected_synonym: ", selected_synonym)
                    syn_dict[chosen_word] = selected_synonym

    # return syn_dict
    # Replace selected words
    for old, new in syn_dict.items():
        text = text.replace(old, new)
    return text

In [304]:
N = 5
for i, data in enumerate(test_dataset):
    sentence = data["text"]
    print("Original\n", sentence)
    sentence_new, syn_dict = replace_with_synonym(sentence, 0.40)
    print("Synonym\n", sentence_new)
    print(syn_dict)
    if i > N:
        break

Original
 I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn't match the background, and painfully one-dimensional characters cannot be overcome with a 'sci-fi' setting. (I'm sure there are those of you out there who think Babylon 5 is good sci-fi TV. It's not. It's clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It's really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it's rubbish as they hav

# Typos

In [18]:
keyboard_layout = {
    'Q': ['W', 'A', 'S'], 'W': ['Q', 'E', 'A', 'S', 'D'], 'E': ['W', 'R', 'S', 'D', 'F'], 'R': ['E', 'T', 'D', 'F', 'G'], 'T': ['R', 'Y', 'F', 'G', 'H'], 
    'Y': ['T', 'U', 'G', 'H', 'J'], 'U': ['Y', 'I', 'H', 'J', 'K'], 'I': ['U', 'O', 'J', 'K', 'L'], 'O': ['I', 'P', 'K', 'L'], 'P': ['O', 'L'],
    'A': ['Q', 'W', 'S', 'Z', 'X'], 'S': ['Q', 'W', 'E', 'A', 'D', 'Z', 'X', 'C'], 'D': ['W', 'E', 'R', 'S', 'F', 'X', 'C', 'V'], 
    'F': ['E', 'R', 'T', 'D', 'G', 'C', 'V', 'B'], 'G': ['R', 'T', 'Y', 'F', 'H', 'V', 'B', 'N'], 'H': ['T', 'Y', 'U', 'G', 'J', 'B', 'N', 'M'], 
    'J': ['Y', 'U', 'I', 'H', 'K', 'N', 'M'], 'K': ['U', 'I', 'O', 'J', 'L', 'M'], 'L': ['I', 'O', 'P', 'K'], 'Z': ['A', 'S', 'X'], 'X': ['Z', 'A', 'S', 'D', 'C'],
    'C': ['X', 'S', 'D', 'F', 'V'], 'V': ['C', 'D', 'F', 'G', 'B'], 'B': ['V', 'F', 'G', 'H', 'N'], 'N': ['B', 'G', 'H', 'J', 'M'], 'M': ['N', 'H', 'J', 'K'],
    'I': ['U', 'O', 'J', 'K', 'L'], 'W': ['Q', 'E', 'A', 'S', 'D']
}

In [214]:
index = random.randint(0, len(word) - 2)

In [19]:
def typo_qwert(word: str, index: int):
    """Replace words close to each other in the QWERT keyboard
    :param word: 
    :param index: index of caracter to be replaced
    """
    # 
    char = word[index]
    if char.upper() in keyboard_layout.keys():
        random_typo = random.choice(keyboard_layout[char.upper()]).lower()
        word = word[:index] + random_typo + word[index+1:]

    return word

In [20]:
def typo_swap_characeters(word: str, index: int):
    """Swap character at index with index + 1
    :param word: 
    :param index: index of caracter to be replaced
    """
    char_list = list(word)
    next_index = index + 1
    char_list[index], char_list[next_index] = char_list[next_index], char_list[index]
    word_swapped = "".join(char_list)
    return word_swapped
    

In [21]:
def typo_remove_character(word: str, index: int):
    """Delete a character
    :param word: 
    :param index: index of caracter to be replaced
    """
    word = word[:index] + word[index + 1:]
    return word

In [22]:
def typo_add_character(word: str, index: int):
    """Add character 
    :param word: 
    :param index: index of caracter to be replaced
    """
    random_char = random.choice(string.ascii_letters)

    return word[:index] + random_char + word[index:]

In [23]:
def add_typos(text: str, pct_typos: float = 0.4, typos_list=['swap', 'remove', 'add', 'qwert']) -> str:
    """Replace {pct_typos}% of words in a sentence with their syntetic generated typo
    :param text (str):
    :param pct_typos (float): percentage of words to be modified. Between 0 and 1
    :return: transformed text
    """
    # Tokenize sentence
    word_list  = word_tokenize(text)


    # word_list words that have at least 3 characters
    word_list_indices_filtered = [i for i in range(len(word_list)) if len(word_list[i]) >= 3]

    n_tokens = len(word_list_indices_filtered) # only count the tokens that have >= characters
    n_typos = int(n_tokens*pct_typos) # number of typos

    # Typos functions
    typos_functions = {
        'swap': typo_swap_characeters,
        'remove': typo_remove_character,
        'add': typo_add_character,
        'qwert': typo_qwert
    }

    # Select indices based on pct_typos
    word_indices = [index for index in range(len(word_list)) if index in word_list_indices_filtered]
    selected_indices = random.sample(word_indices, k = n_typos)

    # Transformed words
    transformed_words = {}
    for index in selected_indices:

        word = word_list[index]
        typo_index= random.randint(0, len(word) - 2)
        # Randomly select typo technique
        selected_typo = random.choice(typos_list)

        # Apply typo function
        word_transformed = typos_functions[selected_typo](word, typo_index)
        transformed_words[index] = word_transformed

    
    # Replace orginal words with sythetic generated typos
    for i, new_word in transformed_words.items():
        word_list[i] = new_word

    # Put sentence together again:
    text = TreebankWordDetokenizer().detokenize(word_list)
    
    
    return text

In [276]:
add_typos(example, 0.6)

'I qlove sci-fi and am willing to pMut up wjth a lot . Si-fi movies/TV xre usualy underfunded, under-appreciated and misuderstood . I tridd to liDke ths, I really dd, but it is to good TV sci-fi as BPabylon 5 is to tar Tek (the Soriginal). SZilly prostheMtics, ceap cardboMard sets, stilted dialogues, CG tyat does nu\'t mach the background, and painfulpy one-dimensional chaarcters cNan not be overcoem wiBth a \'ssci-fi\' detting . (I\'m sjre thee are thlse of you tout there wtho tihnk Babyon 5 is good wci-fi TV . It\'s nlt . It\'s cliczhéd aNnd uninspiring .) Whlie US vewers Bmight like emotin and character developmet, sci-fi is a genre that does nit take tiself seriousyl (cf . Star rek). It may treat important issuse, yt not as a esrious philosophy . It\'s really difficult to crae about hte characters hree as twhey ae not sinmply foolish, just jissing a sprak of Ulife . Teir ations adn reactjons rae wooden and predicatble, often painful to watch . hTe makmers of Ebarth KgOW it\'s rubib

# Expand Contractions

In [27]:
def expand_contradictions(text):

    contraction_mapping = {
        "won't": "will not",
        "can't": "can not",
        "n't": " not",
        "'re": " are",
        "'s": " is",
        "'d": " would",
        "'ll": " will",
        "'ve": " have",
        "'m": " am",
        "won\'t": "will not",
        "can\'t": "can not",
        "n\'t": " not",
        "isn\'t": "is not",
        "\'re": " are",
        "\'s": " is",
        "\'d": " would",
        "\'ll": " will",
        "\'ve": " have",
        "\'m": " am"
    }

    pattern = re.compile(r"\b(?:" + "|".join(re.escape(contraction) for contraction in contraction_mapping.keys()) + r")\b")
    text = pattern.sub(lambda x: contraction_mapping[x.group()], text)
    
    return text

# Random Swap

In [67]:
def random_swap(text: str, pct_swap: float = 0.2) -> str:
    """Swap {pct_swap}% of words in a sentence with a random chosen word
    :param text (str):
    :param pct_swap (float): percentage of words to be swapped. Between 0 and 1
    :return: transformed text
    """
    # Tokenize sentence
    word_list  = word_tokenize(text)

    n_tokens = len(word_list)
    n_swaps = int(n_tokens*pct_swap)
    for i in range(n_swaps):
        # Get indices
        index_1, index_2 = random.sample(range(len(word_list)), 2)
        # Swap words
        word_list[index_1], word_list[index_2] = word_list[index_2], word_list[index_1]


    # Put sentence together again:
    text = TreebankWordDetokenizer().detokenize(word_list)
    return text

In [66]:
print(example)
print(random_swap(example))

Now, I LOVE Italian horror films. The cheesier they are, the better. However, this is not cheesy Italian. This is week-old spaghetti sauce with rotting meatballs. It is amateur hour on every level. There is no suspense, no horror, with just a few drops of blood scattered around to remind you that you are in fact watching a horror film. The "special effects" consist of the lights changing to red whenever the ghost (or whatever it was supposed to be) is around, and a string pulling bed sheets up and down. Oooh, can you feel the chills? The DVD quality is that of a VHS transfer (which actually helps the film more than hurts it). The dubbing is below even the lowest "bad Italian movie" standards and I gave it one star just because the dialogue is so hilarious! And what do we discover when she finally DOES look in the attic (in a scene that is daytime one minute and night the next)...well, I won't spoil it for anyone who really wants to see, but let's just say that it isn't very "novel"!
No

# Random deletion

In [73]:
def random_deletion(text: str, pct_deletion: float = 0.1) -> str:
    """Remove {pct_deletion}% of words in a sentence
    :param text (str):
    :param pct_deletion (float): percentage of words to be deleted. Between 0 and 1
    :return: transformed text
    """
    # Tokenize sentence
    word_list  = word_tokenize(text)

    n_tokens = len(word_list)
    n_deletion = int(n_tokens*pct_deletion)

    indexes_deleted = random.sample(range(len(word_list)), n_deletion)
    for index in sorted(indexes_deleted, reverse=True):
        word_list.pop(index)


    # Put sentence together again:
    text = TreebankWordDetokenizer().detokenize(word_list)
    return text

# Synonym with Word2Vec

In [174]:
from gensim.models import KeyedVectors, Word2Vec
loaded_model = Word2Vec.load("word2vec_model.bin")

In [195]:
def replace_with_synonym_word2vec(text: str, pct_synonyms: float = 0.4, word2vec_file: str = "word2vec_model.bin") -> str:
    """Replace {pct_synonyms}% of words in a sentence with their synonyms based on Word2Vec cosine similarity
    :param text (str):
    :param pct_synonyms (float): percentage of words to be replaced. Between 0 and 1
    :param word2vec_file (str): word2vec file
    :return: transformed text
    """
    # Tokenize sentence
    word_list  = word_tokenize(text)

    # Read word2vec
    loaded_model = Word2Vec.load("word2vec_model.bin")
    vocabulary = loaded_model.wv.index_to_key
    
    # word_list words that have are in vocabulary
    word_list_indices_filtered = [i for i in range(len(word_list)) if word_list[i] in vocabulary]

    n_tokens = len(word_list_indices_filtered) # only count the tokens that are in vocabulary
    n_synonyms = int(n_tokens*pct_synonyms)


    # Select indices based on pct_typos
    word_indices = [index for index in range(len(word_list)) if index in word_list_indices_filtered]
    selected_indices = random.sample(word_indices, k = n_synonyms)

    for chosen_index in selected_indices:
        chosen_word = word_list[chosen_index]

        synonym = loaded_model.wv.most_similar(chosen_word, topn=1)[0][0]
        # print("Original: ", word_list[chosen_index])
        # print("Synonym: ", synonym)
        word_list[chosen_index] = synonym

    # Put sentence together again:
    text = TreebankWordDetokenizer().detokenize(word_list)
    return text

In [196]:
N = 5
for i, data in enumerate(test_dataset):
    sentence = data["text"]
    print("Original\n", sentence)
    sentence_new = replace_with_synonym_word2vec(sentence, 0.4)
    print("Synonym\n", sentence_new)
    if i > N:
        break

Original
 I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn't match the background, and painfully one-dimensional characters cannot be overcome with a 'sci-fi' setting. (I'm sure there are those of you out there who think Babylon 5 is good sci-fi TV. It's not. It's clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It's really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it's rubbish as they hav

# Custom Transform

In [75]:
def custom_transform(example):
    
    ################################
    ##### YOUR CODE BEGINGS HERE ###
    
    # Design and implement the transformation as mentioned in pdf
    # You are free to implement any transformation but the comments at the top roughly describe
    # how you could implement two of them --- synonym replacement and typos.
    
    # You should update example["text"] using your transformation
    
    # Select transformation
    selected_transformation = random.choice(["typo", "synonym", "swap", "deletion"])

    text = example["text"]
    if selected_transformation == "typo":
        text = add_typos(text)
    elif selected_transformation == "synonym":
        text = replace_with_synonym(text)
    elif selected_transformation == "swap":
        text = random_swap(text)
    else:
        text = random_deletion(text)

    # Apply contractions to all of them
    text = expand_contradictions(text)
    example["text"] = text
    ##### YOUR CODE ENDS HERE ######

    
    return example

In [76]:
dataset = load_dataset("imdb")

In [77]:
transformed_dataset = dataset["test"].map(custom_transform, load_from_cache_file=False)     

Map: 100%|██████████| 25000/25000 [01:03<00:00, 394.72 examples/s]


# Synonym with Word2Vec

In [166]:
from gensim.models import KeyedVectors, Word2Vec
from gensim.scripts.glove2word2vec import glove2word2vec
import numpy as np
import gensim


In [164]:
# from gensim.models.word2vec import LineSentence

# # Replace 'path/to/dataset_txt.txt' with the actual path to your text file
# corpus_file_path = 'dataset_txt.txt'

# # Load the corpus from the text file
# corpus = LineSentence(corpus_file_path)

# # Train Word2Vec model
# model = Word2Vec(sentences=corpus, vector_size=100,  window=5, min_count=1, workers=4)

In [167]:
# model.save("word2vec_model.bin")

In [168]:
loaded_model = Word2Vec.load("word2vec_model.bin")

In [156]:
len(vocabulary)

280617

In [172]:
similar_words = loaded_model.wv.most_similar('king', topn=1)

In [173]:
similar_words

[('leader', 0.9081949591636658)]

In [117]:
embedding_file = glove_file
embedding_matrix = dict()
with open(embedding_file, "r") as file:
    for line in file:
        values = line.strip().split()
        word = values[0]
        word_embedding = np.array([float(emb) for emb in values[1:]])
        embedding_matrix[word] = word_embedding

In [96]:
glove_embeddings = embedding_matrix

In [120]:
total_vec = len(glove_embeddings)
vector_size = 50
binary = False
with gensim.utils.smart_open(word2vec_output_file, 'wb') as fout:
    print(total_vec, vector_size)
    fout.write(gensim.utils.to_utf8("%s %s\n" % (total_vec, vector_size)))
    # store in sorted order: most frequent words at the top
    for word, row in tqdm(glove_embeddings.items()):
        if binary:
            row = row.astype(np.float32)
            fout.write(gensim.utils.to_utf8(word) + b" " + row.tostring())
        else:
            fout.write(gensim.utils.to_utf8("%s %s\n" % (word, ' '.join(repr(val) for val in row))))

AttributeError: module 'gensim.utils' has no attribute 'smart_open'

In [97]:
# Create a KeyedVectors model
glove_model = KeyedVectors(vector_size=len(next(iter(glove_embeddings.values()))), dtype='float32')

In [109]:
#  Create a KeyedVectors model
glove_model = KeyedVectors(vector_size=len(next(iter(glove_embeddings.values()))), dtype='float32')

# Load the embeddings from the dictionary into the model
glove_model.key_to_index = {word: idx for idx, word in enumerate(glove_embeddings)}
glove_model.index_to_key = list(glove_embeddings.keys())
glove_model.vectors = {word: glove_embeddings[word].astype(float) for word in glove_embeddings}

In [110]:
# Example usage
similar_words = glove_model.most_similar('queen', topn=5)
print("Similar words to 'king':", similar_words)

TypeError: loop of ufunc does not support argument 0 of type dict which has no callable conjugate method

# Create Augmented Dataloader

In [3]:
from datasets import concatenate_datasets

In [4]:
dataset = load_dataset("imdb")

In [7]:
train_augmented_size = 5000
train_transformed_sample = dataset["train"].shuffle(seed=42).select(range(train_augmented_size))
train_transformed_sample = train_transformed_sample.map(custom_transform, load_from_cache_file=False)                                                    

In [349]:
dataset['train'] = concatenate_datasets([dataset["train"], train_transformed_sample])

/Users/danieldacosta/miniconda3/envs/hack/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [29]:
# Tokenize the input
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [30]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [31]:
global tokenizer

In [351]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Prepare dataset for use by model
tokenized_dataset = tokenized_dataset.remove_columns(["text"])
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format("torch")

# Create dataloaders for iterating over the dataset
train_dataloader = DataLoader(tokenized_dataset["train"], shuffle=True, batch_size=8)

Map: 100%|██████████| 30000/30000 [00:05<00:00, 5114.09 examples/s]


In [41]:
    # Create dataloaders for iterating over the dataset
train_dataloader = DataLoader(tokenized_dataset["train"], shuffle=True, batch_size=8)
    

In [47]:
train_transformed_dataset = concatenate_datasets([dataset["train"], train_transformed_sample])

In [48]:
train_transformed_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 35000
})

In [49]:
# Created a dataladoer for the augmented training dataset
def create_augmented_dataloader(dataset):
    
    ################################
    ##### YOUR CODE BEGINGS HERE ###
    
    # Here, 'dataset' is the original dataset. You should return a dataloader called 'train_dataloader' (with batch size = 8) -- this
    # dataloader will be for the original training split augmented with 5k random transformed examples from the training set.
    # You may want to set load_from_cache_file to False when using dataset maps
    # You may find it helpful to see how the dataloader was created at other place in this code.

    # 5000 randomly transformed examples
    train_augmented_size = 5000
    train_transformed_sample = dataset["train"].shuffle(seed=42).select(range(train_augmented_size))
    train_transformed_sample = train_transformed_sample.map(custom_transform, load_from_cache_file=False) 

    # Augment the training data with 5000 randomly transformed examples to create the new augmented training dataset
    # Final dataset train size: "25,000" + "5,000" = "30,000" 
    train_transformed_dataset = concatenate_datasets([dataset["train"], train_transformed_sample])                                                
    
    train_dataloader = None

    tokenized_dataset = train_transformed_dataset.map(tokenize_function, batched=True)

    # Prepare dataset for use by model
    tokenized_dataset = tokenized_dataset.remove_columns(["text"])
    tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
    tokenized_dataset.set_format("torch")

    # Create dataloaders for iterating over the dataset
    train_dataloader = DataLoader(tokenized_dataset, shuffle=True, batch_size=8)
    
    ##### YOUR CODE ENDS HERE ######
    
    return train_dataloader

In [53]:
train_dataloader = create_augmented_dataloader(dataset)

Map: 100%|██████████| 5000/5000 [00:21<00:00, 236.76 examples/s]
/Users/danieldacosta/miniconda3/envs/hack/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)
Map: 100%|██████████| 30000/30000 [00:05<00:00, 5074.21 examples/s]


In [52]:
dataset = load_dataset("imdb")

In [38]:
train_augmented_size = 5000
train_transformed_sample = dataset["train"].shuffle(seed=42).select(range(train_augmented_size))
train_transformed_sample = train_transformed_sample.map(custom_transform, load_from_cache_file=False) 

# Augment the training data with 5000 randomly transformed examples to create the new augmented training dataset
# Final dataset train size: "25,000" + "5,000" = "30,000" 
dataset['train'] = concatenate_datasets([dataset["train"], train_transformed_sample])       

Map: 100%|██████████| 5000/5000 [00:21<00:00, 228.69 examples/s]
/Users/danieldacosta/miniconda3/envs/hack/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [40]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Prepare dataset for use by model
tokenized_dataset = tokenized_dataset.remove_columns(["text"])
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format("torch")

Map: 100%|██████████| 30000/30000 [00:06<00:00, 4600.29 examples/s]


In [12]:
def create_transformed_dataloader(dataset, debug_transformation):
    
    # Print 5 random transformed examples
    if debug_transformation:
        small_dataset = dataset["test"].shuffle(seed=42).select(range(5))
        small_transformed_dataset = small_dataset.map(custom_transform, load_from_cache_file=False)
        for k in range(5):
            print("Original Example ", str(k))
            print(small_dataset[k])
            print("\n")
            print("Transformed Example ", str(k))
            print(small_transformed_dataset[k])
            print('='*30)

        exit()
      
    
    transformed_dataset = dataset["test"].map(custom_transform, load_from_cache_file=False)                                                    
    transformed_tokenized_dataset = transformed_dataset.map(tokenize_function, batched=True, load_from_cache_file=False)
    transformed_tokenized_dataset = transformed_tokenized_dataset.remove_columns(["text"])
    transformed_tokenized_dataset = transformed_tokenized_dataset.rename_column("label", "labels")
    transformed_tokenized_dataset.set_format("torch")

    transformed_val_dataset = transformed_tokenized_dataset    
    eval_dataloader = DataLoader(transformed_val_dataset, batch_size=8)
    
    return eval_dataloader

# Testing

In [65]:
word_list  = word_tokenize(example)

In [67]:
chosen_word = random.choice(word_list)
synonyms = wn.synsets(chosen_word)

# If chosen word doesn't have a synonym select retry until finding a word that does.
max_retry = len(word_list)
retry = 0 
while (len(synonyms) == 0 and retry <= max_retry):
    chosen_word = random.choice(word_list)
    synonyms = wn.synsets(chosen_word)

In [74]:
# Perform POS tagging
pos_tags = pos_tag(word_list)

# Print the word and its POS tag
chosen_word_tag = [tag for word, tag in pos_tags if word == chosen_word][0]

In [79]:
synonym_list = set()
for syn in synonyms:
    # Only consider synonyms that have the same POS_TAG
    if syn.pos() == chosen_word_wordnet_tag:
        for lemma in syn.lemmas():
            if (lemma.name() != chosen_word) & (lemma):

                synonym_list.add(lemma.name().replace('_', ' ')) # Replace underscores for multi-word synonyms